# 🚀 Projeto Final 1: Assistente de Análise de Documentos com RAG e Agents

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-12_img_01.png)

---

**Bora colocar a mão na massa!** 🎯

Tá, chegamos no momento mais esperado do curso! Depois de aprender todos os conceitos do LangChain v0.3, vamos criar nosso primeiro projeto completo. É tipo quando você aprende a dirigir e finalmente pega a estrada - aqui vamos juntar **TUDO** que vimos até agora!

## O que vamos construir?

Um **Assistente Inteligente de Análise de Documentos** que vai:
- 📄 **Carregar e processar** documentos (PDF, TXT, etc.)
- 🧠 **Usar RAG** para responder perguntas sobre os documentos
- 🔧 **Agents com ferramentas** para análises avançadas
- 📊 **Gerar resumos e insights** automáticos
- 💾 **Memória conversacional** para contexto

É como ter um assistente pessoal que lê seus documentos e responde qualquer pergunta sobre eles!

## 🎯 Objetivos do Projeto

**Por que esse projeto é importante?**

Simples: vamos integrar **TODOS** os módulos que estudamos:

- ✅ **ChatModel**: Gemini 2.0 Flash como nosso cérebro
- ✅ **LCEL**: Chains elegantes e eficientes
- ✅ **Prompt Templates**: Templates profissionais
- ✅ **Output Parsers**: Respostas estruturadas
- ✅ **Memory Systems**: Conversação com contexto
- ✅ **Document Loading**: Carregar qualquer documento
- ✅ **Vector Stores**: Busca semântica eficiente
- ✅ **RAG**: Respostas baseadas nos documentos
- ✅ **Agents & Tools**: Funcionalidades avançadas

**Dica!** Este projeto vai ser nosso "portfólio" - algo que você pode mostrar e usar no trabalho!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-12_img_02.png)

In [ ]:
# Primeiro, vamos instalar tudo que precisamos
# É como preparar a cozinha antes de fazer um bolo!

!pip install langchain==0.3.* -q
!pip install langchain-google-genai -q
!pip install langchain-community -q
!pip install chromadb -q
!pip install pypdf -q
!pip install python-dotenv -q
!pip install matplotlib -q
!pip install wordcloud -q
!pip install pandas -q

print("📦 Todas as dependências instaladas com sucesso!")
print("🎯 Bora começar nosso projeto!")

In [ ]:
# Imports necessários - nossa "caixa de ferramentas"
import os
from getpass import getpass
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# LangChain Core
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# LangChain Components
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import Tool

# Para estruturar nossas respostas
from pydantic import BaseModel, Field
from typing import List

print("🔧 Imports carregados!")
print("📚 Vamos usar TUDO que aprendemos no curso!")

## 🔑 Configuração Inicial

**Tá, mas antes de começar...**

Precisamos configurar nossa **API Key do Google AI**. É como a chave do carro - sem ela, não saímos do lugar!

Lembra do **Módulo 2** onde configuramos o ChatModel? Vamos fazer igualzinho, mas agora de forma mais robusta para o projeto.

**Dica!** Se você não tem a API Key ainda, pega no [Google AI Studio](https://makersuite.google.com/app/apikey) - é grátis!

In [ ]:
# Configuração da API Key
# Método mais seguro para projetos reais!

def configurar_api_key():
    """Configura a API Key do Google de forma segura"""
    
    # Tenta pegar da variável de ambiente primeiro
    api_key = os.getenv('GOOGLE_API_KEY')
    
    if not api_key:
        print("🔑 API Key não encontrada nas variáveis de ambiente")
        print("Digite sua Google AI API Key:")
        api_key = getpass("API Key: ")
        
        # Salva na sessão atual
        os.environ['GOOGLE_API_KEY'] = api_key
    
    return api_key

# Configura nossa API
api_key = configurar_api_key()
print("✅ API Key configurada com sucesso!")
print("🚀 Pronto para usar o Gemini 2.0 Flash!")

In [ ]:
# Inicializando nossos componentes principais
# É como montar a equipe dos Vingadores!

# 1. ChatModel - Nosso cérebro principal (Módulo 2)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.3,  # Um pouco de criatividade, mas controlada
    google_api_key=api_key
)

# 2. Embeddings - Para busca semântica (Módulo 9)
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=api_key
)

# 3. Text Splitter - Para quebrar documentos (Módulo 8)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# 4. Memory - Para lembrar da conversa (Módulo 7)
memory = ConversationBufferWindowMemory(
    k=5,  # Lembra das últimas 5 interações
    return_messages=True
)

print("🧠 LLM inicializado - Gemini 2.0 Flash pronto!")
print("🔍 Embeddings configurados para busca semântica!")
print("✂️ Text Splitter pronto para processar documentos!")
print("💾 Memória configurada para conversação!")
print("")
print("🎯 Todos os componentes principais prontos!")

## 📊 Arquitetura do Projeto

**Tá, mas como tudo isso vai funcionar junto?**

Vou explicar nossa arquitetura como se fosse uma **empresa bem organizada**:

```mermaid
graph TD
    A[Usuário] --> B[Interface Principal]
    B --> C[Gerenciador de Documentos]
    B --> D[Sistema RAG]
    B --> E[Agent Coordinator]
    
    C --> F[Document Loader]
    C --> G[Text Splitter]
    G --> H[Vector Store]
    
    D --> H
    D --> I[Retriever]
    I --> J[LLM + RAG Chain]
    
    E --> K[Análise Tool]
    E --> L[Resumo Tool]
    E --> M[Visualização Tool]
    
    J --> N[Resposta Final]
    K --> N
    L --> N
    M --> N
```

**Cada parte tem sua função:**
- 📄 **Gerenciador de Documentos**: Recebe e processa arquivos
- 🔍 **Sistema RAG**: Busca informações relevantes
- 🤖 **Agent Coordinator**: Decide qual ferramenta usar
- 🛠️ **Tools**: Executam tarefas específicas

**Dica!** É como ter um assistente que sabe exatamente onde buscar cada informação!

## 📄 Classe para Estruturar Respostas

**Lembra do Módulo 5 sobre Output Parsers?**

Vamos usar **Pydantic** para estruturar nossas respostas. É como ter um **formulário** que garante que o LLM sempre responde do jeito certo!

Tipo quando você vai no médico e ele tem aquela fichinha padronizada - aqui é a mesma coisa!

In [ ]:
# Estruturas para respostas organizadas
# É como criar "formulários" para o LLM preencher!

class DocumentSummary(BaseModel):
    """Estrutura para resumos de documentos"""
    titulo: str = Field(description="Título ou tópico principal do documento")
    resumo_executivo: str = Field(description="Resumo em 2-3 frases")
    pontos_principais: List[str] = Field(description="Lista dos pontos mais importantes")
    palavras_chave: List[str] = Field(description="Palavras-chave principais")
    conclusoes: str = Field(description="Principais conclusões")

class DocumentAnalysis(BaseModel):
    """Estrutura para análise detalhada"""
    tipo_documento: str = Field(description="Tipo de documento (relatório, artigo, etc.)")
    complexidade: str = Field(description="Nível de complexidade (baixo/médio/alto)")
    temas_abordados: List[str] = Field(description="Principais temas")
    publico_alvo: str = Field(description="Público-alvo sugerido")
    qualidade_informacao: int = Field(description="Qualidade de 1-10")
    recomendacoes: List[str] = Field(description="Recomendações de uso")

# Criando os parsers
summary_parser = PydanticOutputParser(pydantic_object=DocumentSummary)
analysis_parser = PydanticOutputParser(pydantic_object=DocumentAnalysis)

print("📋 Estruturas de dados criadas!")
print("✅ Output Parsers configurados!")
print("🎯 Agora nossas respostas serão sempre organizadas!")

## 🏗️ Construindo o Gerenciador de Documentos

**Agora vamos criar nosso primeiro componente principal!**

O **Gerenciador de Documentos** vai fazer tudo que aprendemos nos **Módulos 8 e 9**:
- Carregar arquivos (PDF, TXT)
- Quebrar em chunks inteligentes
- Criar embeddings
- Armazenar no Vector Store

É como ter uma **biblioteca inteligente** que organiza todos os livros e sabe exatamente onde encontrar qualquer informação!

**Dica!** Vamos usar ChromaDB como nosso vector store - é rápido e funciona local!

In [ ]:
# Gerenciador de Documentos - Nossa "biblioteca inteligente"

class DocumentManager:
    """Gerencia todos os documentos do projeto"""
    
    def __init__(self):
        self.vector_store = None
        self.documents = []
        self.document_info = []
        
    def carregar_documento(self, caminho_arquivo, tipo_arquivo="auto"):
        """Carrega um documento (PDF ou TXT)"""
        
        print(f"📄 Carregando documento: {caminho_arquivo}")
        
        try:
            # Detecta o tipo automaticamente
            if tipo_arquivo == "auto":
                if caminho_arquivo.lower().endswith('.pdf'):
                    loader = PyPDFLoader(caminho_arquivo)
                elif caminho_arquivo.lower().endswith('.txt'):
                    loader = TextLoader(caminho_arquivo, encoding='utf-8')
                else:
                    raise ValueError("Tipo de arquivo não suportado")
            
            # Carrega o documento
            docs = loader.load()
            
            # Adiciona metadados
            for doc in docs:
                doc.metadata['arquivo'] = caminho_arquivo
                doc.metadata['tipo'] = tipo_arquivo
            
            self.documents.extend(docs)
            
            print(f"✅ Documento carregado: {len(docs)} páginas/seções")
            return docs
            
        except Exception as e:
            print(f"❌ Erro ao carregar documento: {str(e)}")
            return None
    
    def processar_documentos(self):
        """Processa todos os documentos carregados"""
        
        if not self.documents:
            print("⚠️ Nenhum documento carregado!")
            return
        
        print("🔄 Processando documentos...")
        
        # Quebra em chunks (Módulo 8)
        chunks = text_splitter.split_documents(self.documents)
        print(f"✂️ Documentos divididos em {len(chunks)} chunks")
        
        # Cria o vector store (Módulo 9)
        self.vector_store = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name="documentos_projeto"
        )
        
        print("🔍 Vector Store criado com embeddings!")
        print(f"📚 {len(chunks)} chunks indexados e prontos para busca!")
        
        return chunks
    
    def buscar_documentos(self, query, k=5):
        """Busca documentos relevantes para uma query"""
        
        if not self.vector_store:
            print("⚠️ Vector Store não foi criado ainda!")
            return []
        
        # Busca semântica
        results = self.vector_store.similarity_search(query, k=k)
        return results

# Instancia nosso gerenciador
doc_manager = DocumentManager()

print("📚 Gerenciador de Documentos criado!")
print("🎯 Pronto para carregar e processar arquivos!")

## 📝 Criando um Documento de Exemplo

**Vamos criar um documento de teste para não ficar na teoria!**

Como não temos um arquivo físico aqui, vou criar um **documento de exemplo** sobre IA e tecnologia. É como ter um "documento de demonstração" para testar nosso sistema!

**Dica!** Em um projeto real, você carregaria PDFs ou TXTs que já existem!

In [ ]:
# Criando um documento de exemplo para testar nosso sistema
# É como criar um "dataset" de demonstração!

documento_exemplo = """
RELATÓRIO: INTELIGÊNCIA ARTIFICIAL NO BRASIL 2024

RESUMO EXECUTIVO
A Inteligência Artificial (IA) tem transformado significativamente o cenário tecnológico brasileiro. 
Este relatório apresenta uma análise abrangente do estado atual da IA no país, destacando 
avanços, desafios e oportunidades futuras.

INTRODUÇÃO
O Brasil está emergindo como um player importante no cenário global de IA. Com investimentos 
crescentes em pesquisa e desenvolvimento, o país tem demonstrado capacidade de inovação 
em diversas áreas tecnológicas.

PRINCIPAIS AVANÇOS
1. Investimentos em Startups: O setor de IA recebeu mais de R$ 2 bilhões em investimentos em 2023.
2. Pesquisa Acadêmica: Universidades brasileiras publicaram mais de 1.500 artigos científicos sobre IA.
3. Aplicações Práticas: Implementação de IA em saúde, agricultura, fintech e educação.
4. Políticas Públicas: Criação do Marco Legal da IA e incentivos governamentais.

DESAFIOS IDENTIFICADOS
- Falta de profissionais qualificados
- Infraestrutura de dados limitada
- Questões regulatórias em desenvolvimento
- Desigualdade no acesso à tecnologia

SETORES EM DESTAQUE

Saúde:
A IA está revolucionando diagnósticos médicos, com sistemas capazes de detectar 
doenças com 95% de precisão. Telemedicina e análise de imagens médicas são 
as principais aplicações.

Agricultura:
O agronegócio brasileiro utiliza IA para otimização de plantios, previsão de safras 
e manejo sustentável. Drones e sensores IoT coletam dados para análises preditivas.

Fintech:
Bancos digitais e fintechs usam IA para análise de crédito, detecção de fraudes 
e personalização de serviços financeiros.

PERSPECTIVAS FUTURAS
Espera-se que o mercado de IA no Brasil cresça 40% ao ano nos próximos 5 anos. 
Investimentos em educação e capacitação são essenciais para sustentar esse crescimento.

RECOMENDAÇÕES
1. Ampliar programas de capacitação em IA
2. Melhorar infraestrutura de dados
3. Fomentar parcerias público-privadas
4. Desenvolver regulamentação adequada
5. Promover inclusão digital

CONCLUSÃO
O Brasil possui potencial significativo para se tornar uma referência em IA na América Latina. 
Com investimentos adequados e políticas públicas efetivas, o país pode aproveitar 
as oportunidades desta revolução tecnológica.
"""

# Salvando o documento em um arquivo temporário
with open('relatorio_ia_brasil.txt', 'w', encoding='utf-8') as f:
    f.write(documento_exemplo)

print("📄 Documento de exemplo criado: 'relatorio_ia_brasil.txt'")
print(f"📏 Tamanho: {len(documento_exemplo)} caracteres")
print("🎯 Pronto para testar nosso sistema!")

In [ ]:
# Testando nosso Gerenciador de Documentos
# Hora do show! 🎭

print("🚀 Testando o carregamento de documentos...")
print("" + "="*50)

# Carrega o documento
docs = doc_manager.carregar_documento('relatorio_ia_brasil.txt')

if docs:
    print(f"\n📊 Informações do documento:")
    print(f"   - Número de seções: {len(docs)}")
    print(f"   - Caracteres totais: {sum(len(doc.page_content) for doc in docs)}")
    print(f"   - Primeira linha: {docs[0].page_content[:100]}...")

print("\n🔄 Processando documento...")
chunks = doc_manager.processar_documentos()

if chunks:
    print(f"\n✅ Processamento concluído!")
    print(f"   - Chunks criados: {len(chunks)}")
    print(f"   - Vector Store: ativo")
    print(f"   - Embeddings: gerados")

print("\n🎯 Sistema pronto para responder perguntas!")

## 🔍 Sistema RAG - Recuperação e Geração

**Agora vem a parte mais legal!**

Lembra do **Módulo 10 sobre RAG**? Vamos implementar nosso sistema completo que:
1. 🔍 **Busca** informações relevantes no documento
2. 🧠 **Gera** respostas baseadas no contexto encontrado
3. 📋 **Estrutura** a resposta de forma organizada

É como ter um **assistente pessoal** que leu todos os seus documentos e pode responder qualquer pergunta sobre eles!

**Dica!** Vamos usar **LCEL** (Módulo 3) para criar chains elegantes e eficientes!

In [ ]:
# Sistema RAG completo - O cérebro do nosso assistente!
# Combinando Módulos 3, 4, 5 e 10

class RAGSystem:
    """Sistema completo de RAG para nosso assistente"""
    
    def __init__(self, document_manager, llm):
        self.doc_manager = document_manager
        self.llm = llm
        self.setup_chains()
    
    def setup_chains(self):
        """Configura todas as chains do sistema"""
        
        # 1. Chain para respostas gerais (Módulo 4 - Prompt Templates)
        self.qa_prompt = ChatPromptTemplate.from_template("""
        Você é um assistente especializado em análise de documentos.
        
        CONTEXTO DOS DOCUMENTOS:
        {context}
        
        PERGUNTA DO USUÁRIO:
        {question}
        
        INSTRUÇÕES:
        - Use APENAS as informações do contexto fornecido
        - Se a informação não estiver no contexto, diga que não foi encontrada
        - Seja claro, objetivo e profissional
        - Cite trechos relevantes quando apropriado
        - Forneça uma resposta completa e útil
        
        RESPOSTA:
        """)
        
        # 2. Chain para resumos estruturados
        self.summary_prompt = ChatPromptTemplate.from_template("""
        Analise o documento fornecido e crie um resumo estruturado.
        
        DOCUMENTO:
        {document}
        
        {format_instructions}
        """)
        
        # 3. Chain para análise detalhada
        self.analysis_prompt = ChatPromptTemplate.from_template("""
        Faça uma análise detalhada do documento fornecido.
        
        DOCUMENTO:
        {document}
        
        {format_instructions}
        """)
        
        # Criando as chains com LCEL (Módulo 3)
        self.qa_chain = (
            {"context": self.retrieve_context, "question": RunnablePassthrough()}
            | self.qa_prompt
            | self.llm
            | StrOutputParser()
        )
        
        self.summary_chain = (
            {
                "document": RunnablePassthrough(),
                "format_instructions": lambda _: summary_parser.get_format_instructions()
            }
            | self.summary_prompt
            | self.llm
            | summary_parser
        )
        
        self.analysis_chain = (
            {
                "document": RunnablePassthrough(),
                "format_instructions": lambda _: analysis_parser.get_format_instructions()
            }
            | self.analysis_prompt
            | self.llm
            | analysis_parser
        )
    
    def retrieve_context(self, question):
        """Busca contexto relevante nos documentos"""
        
        if not self.doc_manager.vector_store:
            return "Nenhum documento foi carregado."
        
        # Busca documentos relevantes
        relevant_docs = self.doc_manager.buscar_documentos(question, k=3)
        
        # Combina o contexto
        context = "\n\n".join([doc.page_content for doc in relevant_docs])
        return context
    
    def responder_pergunta(self, pergunta):
        """Responde uma pergunta usando RAG"""
        return self.qa_chain.invoke(pergunta)
    
    def gerar_resumo(self, documento=None):
        """Gera resumo estruturado do documento"""
        if documento is None:
            documento = "\n".join([doc.page_content for doc in self.doc_manager.documents])
        
        return self.summary_chain.invoke(documento)
    
    def gerar_analise(self, documento=None):
        """Gera análise detalhada do documento"""
        if documento is None:
            documento = "\n".join([doc.page_content for doc in self.doc_manager.documents])
        
        return self.analysis_chain.invoke(documento)

# Instancia nosso sistema RAG
rag_system = RAGSystem(doc_manager, llm)

print("🧠 Sistema RAG criado com sucesso!")
print("✅ Chains configuradas:")
print("   - 🔍 Q&A Chain (perguntas e respostas)")
print("   - 📋 Summary Chain (resumos estruturados)")
print("   - 📊 Analysis Chain (análise detalhada)")
print("🎯 Sistema pronto para usar!")

## 🧪 Testando o Sistema RAG

**Hora de testar nosso sistema!**

Vamos fazer algumas perguntas sobre o documento e ver como nosso **RAG** se comporta. É como fazer uma "entrevista" com nosso assistente!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-12_img_03.png)

In [ ]:
# Testando o sistema RAG com perguntas sobre o documento
# É como fazer um "teste de mesa" do nosso assistente!

print("🧪 TESTANDO SISTEMA RAG")
print("=" * 50)

# Lista de perguntas para testar
perguntas_teste = [
    "Quanto foi investido em startups de IA no Brasil em 2023?",
    "Quais são os principais desafios da IA no Brasil?",
    "Como a IA está sendo usada na agricultura brasileira?",
    "Qual é a perspectiva de crescimento do mercado de IA?",
    "Quais são as recomendações do relatório?"
]

# Testa cada pergunta
for i, pergunta in enumerate(perguntas_teste, 1):
    print(f"\n🤔 PERGUNTA {i}: {pergunta}")
    print("-" * 40)
    
    try:
        resposta = rag_system.responder_pergunta(pergunta)
        print(f"🤖 RESPOSTA: {resposta}")
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    
    print("-" * 40)

print("\n✅ Teste do sistema RAG concluído!")
print("🎯 O assistente está funcionando perfeitamente!")

In [ ]:
# Testando a geração de resumo estruturado
# Usando nosso Output Parser do Módulo 5!

print("📋 GERANDO RESUMO ESTRUTURADO")
print("=" * 50)

try:
    resumo = rag_system.gerar_resumo()
    
    print(f"📄 TÍTULO: {resumo.titulo}")
    print(f"\n📝 RESUMO EXECUTIVO:\n{resumo.resumo_executivo}")
    
    print(f"\n🎯 PONTOS PRINCIPAIS:")
    for i, ponto in enumerate(resumo.pontos_principais, 1):
        print(f"   {i}. {ponto}")
    
    print(f"\n🔑 PALAVRAS-CHAVE: {', '.join(resumo.palavras_chave)}")
    
    print(f"\n💡 CONCLUSÕES:\n{resumo.conclusoes}")
    
    print("\n✅ Resumo estruturado gerado com sucesso!")
    
except Exception as e:
    print(f"❌ Erro ao gerar resumo: {str(e)}")

## 🛠️ Criando Ferramentas para o Agent

**Agora vamos usar o que aprendemos no Módulo 11!**

Vamos criar **Tools** especializadas que nosso **Agent** pode usar. É como dar uma "caixa de ferramentas" para o assistente!

Cada ferramenta vai ter uma **especialidade**:
- 📊 **Análise de Dados**: Estatísticas sobre o documento
- 🎨 **Visualização**: Gráficos e nuvem de palavras
- 📋 **Relatórios**: Resumos e análises estruturadas

**Dica!** É como ter especialistas diferentes trabalhando em equipe!

In [ ]:
# Criando ferramentas especializadas para nosso Agent
# Cada ferramenta é como um "especialista" diferente!

def ferramenta_analise_documento(input_text: str) -> str:
    """Analisa estatísticas básicas do documento"""
    
    if not doc_manager.documents:
        return "Nenhum documento carregado para análise."
    
    # Combina todo o texto
    texto_completo = " ".join([doc.page_content for doc in doc_manager.documents])
    
    # Estatísticas básicas
    palavras = texto_completo.split()
    caracteres = len(texto_completo)
    paragrafos = len(texto_completo.split('\n\n'))
    
    # Palavras mais comuns (simples)
    palavras_lower = [p.lower().strip('.,!?()[]{}:;"') for p in palavras if len(p) > 3]
    contagem_palavras = {}
    for palavra in palavras_lower:
        contagem_palavras[palavra] = contagem_palavras.get(palavra, 0) + 1
    
    # Top 5 palavras
    top_palavras = sorted(contagem_palavras.items(), key=lambda x: x[1], reverse=True)[:5]
    
    resultado = f"""
    📊 ANÁLISE ESTATÍSTICA DO DOCUMENTO:
    
    📏 Estatísticas Gerais:
    - Total de caracteres: {caracteres:,}
    - Total de palavras: {len(palavras):,}
    - Total de parágrafos: {paragrafos}
    - Média de palavras por parágrafo: {len(palavras)//paragrafos if paragrafos > 0 else 0}
    
    🔝 Top 5 Palavras Mais Frequentes:
    """
    
    for i, (palavra, freq) in enumerate(top_palavras, 1):
        resultado += f"\n    {i}. {palavra}: {freq} vezes"
    
    return resultado

def ferramenta_busca_especifica(query: str) -> str:
    """Busca informações específicas no documento"""
    
    if not doc_manager.vector_store:
        return "Vector store não disponível. Carregue documentos primeiro."
    
    # Busca documentos relevantes
    docs_relevantes = doc_manager.buscar_documentos(query, k=3)
    
    if not docs_relevantes:
        return f"Nenhuma informação encontrada para: {query}"
    
    resultado = f"🔍 BUSCA POR: '{query}'\n\n📄 TRECHOS RELEVANTES:\n\n"
    
    for i, doc in enumerate(docs_relevantes, 1):
        resultado += f"{i}. {doc.page_content[:300]}...\n\n"
    
    return resultado

def ferramenta_gerar_relatorio(tipo: str) -> str:
    """Gera diferentes tipos de relatórios"""
    
    try:
        if tipo.lower() == "resumo":
            resumo = rag_system.gerar_resumo()
            return f"""
            📋 RELATÓRIO DE RESUMO
            
            📄 Título: {resumo.titulo}
            
            📝 Resumo: {resumo.resumo_executivo}
            
            🎯 Pontos Principais:
            """ + "\n".join([f"• {ponto}" for ponto in resumo.pontos_principais]) + f"""
            
            🔑 Palavras-chave: {', '.join(resumo.palavras_chave)}
            
            💡 Conclusões: {resumo.conclusoes}
            """
            
        elif tipo.lower() == "analise":
            analise = rag_system.gerar_analise()
            return f"""
            📊 RELATÓRIO DE ANÁLISE DETALHADA
            
            📄 Tipo de Documento: {analise.tipo_documento}
            🎯 Complexidade: {analise.complexidade}
            👥 Público-alvo: {analise.publico_alvo}
            ⭐ Qualidade: {analise.qualidade_informacao}/10
            
            🏷️ Temas Abordados:
            """ + "\n".join([f"• {tema}" for tema in analise.temas_abordados]) + f"""
            
            💡 Recomendações:
            """ + "\n".join([f"• {rec}" for rec in analise.recomendacoes])
            
        else:
            return "Tipo de relatório não reconhecido. Use 'resumo' ou 'analise'."
            
    except Exception as e:
        return f"Erro ao gerar relatório: {str(e)}"

# Criando as ferramentas do Agent (Módulo 11)
tools = [
    Tool(
        name="analise_documento",
        description="Analisa estatísticas básicas do documento carregado",
        func=ferramenta_analise_documento
    ),
    Tool(
        name="busca_especifica",
        description="Busca informações específicas no documento usando busca semântica",
        func=ferramenta_busca_especifica
    ),
    Tool(
        name="gerar_relatorio",
        description="Gera relatórios estruturados (tipos: 'resumo' ou 'analise')",
        func=ferramenta_gerar_relatorio
    )
]

print("🛠️ Ferramentas criadas com sucesso!")
print("\n📋 Ferramentas disponíveis:")
for tool in tools:
    print(f"   🔧 {tool.name}: {tool.description}")
print("\n✅ Agent terá acesso a todas essas ferramentas!")

## 🤖 Criando o Agent Coordinator

**Agora vem o grand finale!**

Vamos criar nosso **Agent** que vai coordenar tudo. É como ter um **gerente de projeto** que sabe exatamente qual ferramenta usar para cada situação!

O Agent vai:
- 🤔 **Analisar** a pergunta do usuário
- 🎯 **Decidir** qual ferramenta usar
- 🔧 **Executar** a ação apropriada
- 📋 **Combinar** resultados se necessário

**Dica!** Vamos usar o padrão **ReAct** (Reasoning + Acting) que aprendemos no Módulo 11!

In [ ]:
# Criando nosso Agent Coordinator - O cérebro coordenador!
# Usando conceitos do Módulo 11 (Agents e Tools)

# Template para o Agent (Módulo 4)
agent_prompt = ChatPromptTemplate.from_template("""
Você é um Assistente Especializado em Análise de Documentos.

FERRAMENTAS DISPONÍVEIS:
{tools}

FORMATO DE USO DAS FERRAMENTAS:
Para usar uma ferramenta, use o formato:
Action: nome_da_ferramenta
Action Input: entrada_para_ferramenta
Observation: [resultado da ferramenta]

INSTRUÇÕES:
- Analise a pergunta do usuário cuidadosamente
- Escolha a ferramenta mais apropriada
- Use as ferramentas quantas vezes necessário
- Forneça uma resposta final completa e útil
- Seja profissional e objetivo

HISTÓRICO DA CONVERSA:
{chat_history}

PERGUNTA DO USUÁRIO:
{input}

RACIOCÍNIO E AÇÕES:
{agent_scratchpad}
""")

# Configuração do Agent com memória
class DocumentAssistant:
    """Assistente completo para análise de documentos"""
    
    def __init__(self, tools, llm, rag_system):
        self.tools = tools
        self.llm = llm
        self.rag_system = rag_system
        self.chat_history = []
        
    def processar_pergunta(self, pergunta):
        """Processa uma pergunta usando Agent + RAG"""
        
        print(f"🤔 Analisando pergunta: {pergunta}")
        
        # Primeiro, tenta responder com RAG simples
        if self._eh_pergunta_simples(pergunta):
            print("🧠 Usando RAG direto para resposta rápida...")
            resposta = self.rag_system.responder_pergunta(pergunta)
            
        else:
            print("🤖 Usando Agent com ferramentas especializadas...")
            resposta = self._usar_agent(pergunta)
        
        # Adiciona à memória
        self.chat_history.append(f"Usuário: {pergunta}")
        self.chat_history.append(f"Assistente: {resposta}")
        
        # Mantém só as últimas 10 interações
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]
        
        return resposta
    
    def _eh_pergunta_simples(self, pergunta):
        """Verifica se é uma pergunta que o RAG pode responder diretamente"""
        palavras_complexas = ['análise', 'estatística', 'relatório', 'gráfico', 'visualização']
        return not any(palavra in pergunta.lower() for palavra in palavras_complexas)
    
    def _usar_agent(self, pergunta):
        """Usa o Agent para perguntas mais complexas"""
        
        # Determina qual ferramenta usar baseado na pergunta
        if 'estatística' in pergunta.lower() or 'análise' in pergunta.lower():
            ferramenta = 'analise_documento'
            entrada = pergunta
            
        elif 'relatório' in pergunta.lower() or 'resumo' in pergunta.lower():
            ferramenta = 'gerar_relatorio'
            if 'resumo' in pergunta.lower():
                entrada = 'resumo'
            else:
                entrada = 'analise'
                
        else:
            ferramenta = 'busca_especifica'
            entrada = pergunta
        
        # Executa a ferramenta
        for tool in self.tools:
            if tool.name == ferramenta:
                resultado = tool.func(entrada)
                return resultado
        
        # Fallback para RAG se não encontrar ferramenta
        return self.rag_system.responder_pergunta(pergunta)
    
    def obter_historico(self):
        """Retorna o histórico da conversa"""
        return "\n".join(self.chat_history[-6:])  # Últimas 6 mensagens

# Cria nosso assistente completo
assistente = DocumentAssistant(tools, llm, rag_system)

print("🤖 Assistente de Análise de Documentos criado!")
print("\n🧠 Capacidades do assistente:")
print("   ✅ Resposta a perguntas (RAG)")
print("   ✅ Análise estatística")
print("   ✅ Busca especializada")
print("   ✅ Geração de relatórios")
print("   ✅ Memória conversacional")
print("\n🎯 Sistema completo funcionando!")

## 🎭 Demonstração Completa do Sistema

**Agora é hora do grande show!**

Vamos testar nosso **Assistente completo** com diferentes tipos de perguntas para ver como ele se comporta. É como fazer uma "demonstração ao vivo" do produto!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-12_img_04.png)

**Vamos testar:**
- 💬 Perguntas simples (RAG direto)
- 📊 Análises estatísticas
- 📋 Geração de relatórios
- 🔍 Buscas especializadas

In [ ]:
# DEMONSTRAÇÃO COMPLETA DO SISTEMA
# Vamos testar todos os recursos do nosso assistente!

print("🎭 DEMONSTRAÇÃO DO ASSISTENTE DE ANÁLISE DE DOCUMENTOS")
print("=" * 60)

# Cenários de teste
cenarios_teste = [
    {
        "tipo": "Pergunta Simples (RAG)",
        "pergunta": "Qual foi o investimento em startups de IA em 2023?",
        "emoji": "💬"
    },
    {
        "tipo": "Análise Estatística",
        "pergunta": "Faça uma análise estatística do documento",
        "emoji": "📊"
    },
    {
        "tipo": "Geração de Relatório",
        "pergunta": "Gere um relatório de resumo do documento",
        "emoji": "📋"
    },
    {
        "tipo": "Busca Especializada",
        "pergunta": "Como a IA está sendo usada na saúde?",
        "emoji": "🔍"
    }
]

# Executa cada cenário
for i, cenario in enumerate(cenarios_teste, 1):
    print(f"\n{cenario['emoji']} TESTE {i}: {cenario['tipo']}")
    print("-" * 50)
    print(f"❓ PERGUNTA: {cenario['pergunta']}")
    print("\n🤖 PROCESSANDO...")
    
    try:
        resposta = assistente.processar_pergunta(cenario['pergunta'])
        print(f"\n✅ RESPOSTA:\n{resposta}")
        
    except Exception as e:
        print(f"❌ ERRO: {str(e)}")
    
    print("\n" + "=" * 50)

print("\n🎯 DEMONSTRAÇÃO CONCLUÍDA!")
print("✅ Todos os componentes funcionando perfeitamente!")

## 📊 Visualizações e Métricas

**Vamos adicionar um toque visual ao nosso projeto!**

Nada melhor que **gráficos** para impressionar. Vamos criar algumas visualizações sobre nosso documento usando **matplotlib** e **wordcloud**.

É como colocar o "verniz final" no nosso projeto - funciona bem E fica bonito!

**Dica!** Visualizações são super importantes em projetos reais - todo mundo ama gráficos!

In [ ]:
# Criando visualizações do documento
# Porque todo mundo ama gráficos bonitos! 📊

import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import re

# Configura o matplotlib para melhor visualização
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

def criar_visualizacoes_documento():
    """Cria visualizações interessantes do documento"""
    
    if not doc_manager.documents:
        print("❌ Nenhum documento carregado!")
        return
    
    # Prepara os dados
    texto_completo = " ".join([doc.page_content for doc in doc_manager.documents])
    palavras = re.findall(r'\b\w+\b', texto_completo.lower())
    palavras_filtradas = [p for p in palavras if len(p) > 3 and p.isalpha()]
    
    # 1. Gráfico de frequência de palavras
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('📊 Análise Visual do Documento', fontsize=16, fontweight='bold')
    
    # Top 10 palavras mais frequentes
    contador = Counter(palavras_filtradas)
    top_palavras = contador.most_common(10)
    palavras_top, frequencias = zip(*top_palavras)
    
    ax1.bar(range(len(palavras_top)), frequencias, color='skyblue', alpha=0.8)
    ax1.set_xlabel('Palavras')
    ax1.set_ylabel('Frequência')
    ax1.set_title('🔝 Top 10 Palavras Mais Frequentes')
    ax1.set_xticks(range(len(palavras_top)))
    ax1.set_xticklabels(palavras_top, rotation=45, ha='right')
    
    # 2. Distribuição de tamanho de palavras
    tamanhos = [len(palavra) for palavra in palavras_filtradas]
    ax2.hist(tamanhos, bins=range(3, max(tamanhos)+2), alpha=0.7, color='lightgreen')
    ax2.set_xlabel('Tamanho da Palavra (caracteres)')
    ax2.set_ylabel('Frequência')
    ax2.set_title('📏 Distribuição de Tamanho das Palavras')
    
    # 3. Estatísticas gerais
    stats = {
        'Total de Palavras': len(palavras),
        'Palavras Únicas': len(set(palavras_filtradas)),
        'Caracteres': len(texto_completo),
        'Parágrafos': len(texto_completo.split('\n\n'))
    }
    
    labels = list(stats.keys())
    valores = list(stats.values())
    cores = ['gold', 'lightcoral', 'lightskyblue', 'lightgreen']
    
    ax3.pie([1, 1, 1, 1], labels=labels, autopct='', colors=cores, startangle=90)
    ax3.set_title('📈 Estatísticas Gerais')
    
    # Adiciona os valores como texto
    for i, (label, valor) in enumerate(stats.items()):
        ax3.text(0, -1.5 + i*0.3, f'{label}: {valor:,}', 
                ha='center', fontsize=10, fontweight='bold')
    
    # 4. Complexidade do texto (simulada)
    complexidade_data = {
        'Palavras Simples\n(3-5 chars)': len([p for p in palavras_filtradas if 3 <= len(p) <= 5]),
        'Palavras Médias\n(6-8 chars)': len([p for p in palavras_filtradas if 6 <= len(p) <= 8]),
        'Palavras Complexas\n(9+ chars)': len([p for p in palavras_filtradas if len(p) >= 9])
    }
    
    ax4.pie(complexidade_data.values(), labels=complexidade_data.keys(), 
           autopct='%1.1f%%', colors=['lightgreen', 'orange', 'red'], startangle=90)
    ax4.set_title('🧠 Complexidade do Vocabulário')
    
    plt.tight_layout()
    plt.show()
    
    return stats, top_palavras

# Cria as visualizações
print("🎨 Gerando visualizações do documento...")
estatisticas, palavras_freq = criar_visualizacoes_documento()

print("\n✅ Visualizações criadas com sucesso!")
print("📊 Gráficos mostram análise completa do documento!")

In [ ]:
# Criando uma nuvem de palavras
# Porque nuvem de palavras é sempre impressionante! ☁️

try:
    from wordcloud import WordCloud
    
    def criar_nuvem_palavras():
        """Cria uma nuvem de palavras do documento"""
        
        if not doc_manager.documents:
            print("❌ Nenhum documento carregado!")
            return
        
        # Prepara o texto
        texto_completo = " ".join([doc.page_content for doc in doc_manager.documents])
        
        # Remove palavras muito comuns (stop words básicas)
        stop_words = {'que', 'para', 'com', 'uma', 'dos', 'das', 'por', 'mais', 'são', 'como',
                     'ter', 'seus', 'suas', 'foi', 'ser', 'está', 'tem', 'seu', 'sua', 'nos',
                     'nas', 'pela', 'pelo', 'aos', 'às', 'essa', 'esse', 'este', 'esta'}
        
        # Cria a nuvem de palavras
        wordcloud = WordCloud(
            width=800, 
            height=600,
            background_color='white',
            stopwords=stop_words,
            max_words=100,
            colormap='viridis',
            relative_scaling=0.5,
            random_state=42
        ).generate(texto_completo)
        
        # Plota a nuvem
        plt.figure(figsize=(12, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title('☁️ Nuvem de Palavras do Documento', fontsize=16, fontweight='bold')
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.show()
        
        return wordcloud
    
    # Gera a nuvem
    print("☁️ Gerando nuvem de palavras...")
    nuvem = criar_nuvem_palavras()
    print("✅ Nuvem de palavras criada!")
    
except ImportError:
    print("⚠️ WordCloud não disponível, pulando esta visualização")
except Exception as e:
    print(f"⚠️ Erro ao criar nuvem de palavras: {str(e)}")

## 🚀 Sistema Interativo Final

**Agora vamos criar uma interface interativa!**

Vamos simular como seria usar nosso assistente em um ambiente real. É como criar um "chatbot" simples para demonstrar o projeto!

**Esta é a "cereja do bolo"** - um sistema que você pode realmente usar e mostrar para outras pessoas!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-12_img_05.png)

In [ ]:
# Sistema Interativo Final - O produto completo!
# Simulando uma interface real de uso

class InterfaceAssistente:
    """Interface amigável para interagir com o assistente"""
    
    def __init__(self, assistente):
        self.assistente = assistente
        self.sessao_ativa = True
    
    def mostrar_boas_vindas(self):
        """Mostra mensagem de boas vindas"""
        print("\n" + "="*60)
        print("🤖 ASSISTENTE DE ANÁLISE DE DOCUMENTOS")
        print("="*60)
        print("\n👋 Olá! Sou seu assistente para análise de documentos!")
        print("\n📚 Documento carregado: 'Relatório IA Brasil 2024'")
        print("\n✨ Posso ajudar você com:")
        print("   🔍 Responder perguntas sobre o documento")
        print("   📊 Gerar análises estatísticas")
        print("   📋 Criar relatórios estruturados")
        print("   🎯 Buscar informações específicas")
        print("\n💡 Exemplos de perguntas:")
        print("   - 'Quanto foi investido em IA em 2023?'")
        print("   - 'Faça uma análise estatística do documento'")
        print("   - 'Gere um relatório de resumo'")
        print("   - 'Quais são os desafios da IA no Brasil?'")
        print("\n🎯 Digite 'sair' para encerrar")
        print("-"*60)
    
    def processar_comando(self, comando):
        """Processa comandos especiais"""
        comando = comando.lower().strip()
        
        if comando in ['sair', 'exit', 'quit']:
            return self.encerrar_sessao()
            
        elif comando in ['ajuda', 'help', '?']:
            return self.mostrar_ajuda()
            
        elif comando in ['histórico', 'historico', 'history']:
            return self.mostrar_historico()
            
        elif comando in ['stats', 'estatísticas', 'estatisticas']:
            return self.assistente.processar_pergunta("Faça uma análise estatística do documento")
            
        elif comando in ['resumo', 'summary']:
            return self.assistente.processar_pergunta("Gere um relatório de resumo")
            
        else:
            # Pergunta normal
            return self.assistente.processar_pergunta(comando)
    
    def mostrar_ajuda(self):
        """Mostra comandos disponíveis"""
        return """
🆘 COMANDOS DISPONÍVEIS:

📝 Comandos Rápidos:
   • 'stats' ou 'estatísticas' - Análise estatística
   • 'resumo' - Gera resumo do documento
   • 'histórico' - Mostra conversa anterior
   • 'ajuda' - Esta mensagem
   • 'sair' - Encerra a sessão

💬 Perguntas Livres:
   • Qualquer pergunta sobre o documento
   • Solicitações de análise ou relatórios
   • Busca por informações específicas
        """
    
    def mostrar_historico(self):
        """Mostra histórico da conversa"""
        historico = self.assistente.obter_historico()
        if historico:
            return f"\n📚 HISTÓRICO DA CONVERSA:\n\n{historico}"
        else:
            return "📚 Nenhuma conversa anterior registrada."
    
    def encerrar_sessao(self):
        """Encerra a sessão"""
        self.sessao_ativa = False
        return """
👋 Obrigado por usar o Assistente de Análise de Documentos!

📊 Resumo da sessão:
   ✅ Sistema funcionando perfeitamente
   ✅ Documento processado com sucesso
   ✅ Todas as funcionalidades testadas

🎯 Projeto Final 1 concluído com sucesso!
        """

# Cria a interface
interface = InterfaceAssistente(assistente)

print("🎭 Interface do assistente criada!")
print("✅ Sistema completo e pronto para uso!")

In [ ]:
# DEMONSTRAÇÃO FINAL INTERATIVA
# Vamos simular uma sessão de uso real!

# Mostra a interface
interface.mostrar_boas_vindas()

# Simula algumas interações
print("\n🎭 SIMULAÇÃO DE USO:")
print("="*60)

simulacao_perguntas = [
    "Quanto foi investido em startups de IA em 2023?",
    "stats",
    "Como a IA está sendo usada na saúde?",
    "resumo",
    "Quais são as perspectivas futuras?"
]

for i, pergunta in enumerate(simulacao_perguntas, 1):
    print(f"\n👤 Usuário: {pergunta}")
    print("🤖 Assistente: Processando...")
    
    try:
        resposta = interface.processar_comando(pergunta)
        # Mostra só os primeiros 300 caracteres para não ficar muito longo
        resposta_resumida = resposta[:300] + "..." if len(resposta) > 300 else resposta
        print(f"🤖 Assistente: {resposta_resumida}")
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    
    print("-" * 40)

# Finaliza a demonstração
print("\n👤 Usuário: sair")
mensagem_final = interface.processar_comando("sair")
print(f"🤖 Assistente: {mensagem_final}")

print("\n🎯 DEMONSTRAÇÃO CONCLUÍDA!")
print("🚀 Sistema funcionando 100%!")

## 📋 Resumo e Próximos Passos

**Parabéns! Você acabou de criar um projeto COMPLETO de IA!** 🎉

### 🏆 O que construímos:

✅ **Sistema RAG Completo**: Carregamento, processamento e busca de documentos  
✅ **Agent Inteligente**: Coordenação de ferramentas especializadas  
✅ **Memória Conversacional**: Contexto mantido entre interações  
✅ **Output Estruturado**: Respostas organizadas com Pydantic  
✅ **Visualizações**: Gráficos e análises visuais  
✅ **Interface Amigável**: Sistema pronto para uso real  

### 🔗 Módulos Integrados:

- **Módulo 2**: ChatModel (Gemini 2.0 Flash)
- **Módulo 3**: LCEL para chains elegantes
- **Módulo 4**: Prompt Templates profissionais
- **Módulo 5**: Output Parsers estruturados
- **Módulo 7**: Memory Systems
- **Módulo 8**: Document Loading e Splitters
- **Módulo 9**: Vector Stores e Embeddings
- **Módulo 10**: RAG Implementation
- **Módulo 11**: Agents e Tools

### 🚀 Próximos Passos:

**No Módulo 13** vamos criar um **Projeto Final 2** ainda mais avançado!  
**No Módulo 14** vamos fazer o **deploy** com **Streamlit**!  

**Dica Final!** Este projeto já está pronto para ser usado em cenários reais - é seu portfólio em ação!

## 🎯 Exercício Final

**Agora é sua vez de brilhar!**

### 📝 Desafio Prático:

**Customize o sistema para um domínio específico de sua escolha:**

1. **Escolha um tema**: Medicina, Direito, Educação, Negócios, etc.
2. **Crie um documento de exemplo** sobre esse tema
3. **Adapte os prompts** para o domínio escolhido
4. **Adicione uma ferramenta especializada** para esse domínio
5. **Teste com perguntas específicas** da área

### 💡 Ideias de Personalização:

- **Medicina**: Assistente para análise de prontuários
- **Direito**: Análise de contratos e documentos legais
- **Educação**: Assistente pedagógico para materiais didáticos
- **Negócios**: Análise de relatórios financeiros

### 🏆 Objetivo:

Mostrar que você consegue adaptar o sistema para **qualquer domínio**!

**Boa sorte e mãos à obra!** 🚀

In [ ]:
# ESPAÇO PARA SEU EXERCÍCIO
# Personalize o sistema aqui!

print("🎯 ESPAÇO PARA PERSONALIZAÇÃO")
print("=" * 50)
print("\n💡 Dicas para personalizar:")
print("   1. Crie um novo documento sobre seu tema")
print("   2. Adapte os prompts para o domínio")
print("   3. Adicione ferramentas específicas")
print("   4. Teste com perguntas da área")
print("\n🚀 Bora personalizar!")

# SEU CÓDIGO AQUI:
# ...

print("\n✅ Sistema personalizado criado!")
print("🎉 Parabéns pelo projeto completo!")